In [1]:
import pickle
import numpy as np
import torch
import warnings
from functools import reduce
import operator
import matplotlib.pyplot as plt
import torch.nn as nn
import os
torch.xpu.is_available()
warnings.filterwarnings('ignore')

data_path = os.path.join('datasets', 'dc_wss', 'dc_wss_dataset_dc5_ex500')

with open(data_path, 'rb') as f:
    data = pickle.load(f)

file_path = os.path.join('models', 'model_2025-03-29_02-29-15_dcwss_samples500_epochs100.pt')

In [2]:
class NNSolver(nn.Module):
    def __init__(self, data, args):
        super().__init__()
        self._data = data
        self._args = args
        layer_sizes = [data.xdim, self._args['hiddenSize'], self._args['hiddenSize']]
        layers = reduce(operator.add,
            [[nn.Linear(a,b), nn.BatchNorm1d(b), nn.ReLU(), nn.Dropout(p=0.2)]
                for a,b in zip(layer_sizes[0:-1], layer_sizes[1:])])
        
        output_dim = data.ydim - data.nknowns        

        if self._args['useCompl']:
            layers += [nn.Linear(layer_sizes[-1], output_dim - data.neq)]            
        else:
            layers += [nn.Linear(layer_sizes[-1], output_dim)] 
            
        for layer in layers:
            if type(layer) is nn.Linear:
                nn.init.kaiming_normal_(layer.weight)

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        #print('X ', x[0])
        out = self.net(x)

        if self._args['useCompl']:
            result = self._data.complete_partial(x, out)
            return result
        else:
            
            if self._args['probType'] == 'dc_wss':
                out = nn.Sigmoid()(out)
            
            result = self._data.process_output(x, out)
            #print('Out ', out[0])
            return result

In [3]:
args = {'probType': 'dc_wss', 'hiddenSize': 200, 'useCompl': False, 'corrMode': 'full'}

newModel = NNSolver(data, args)

newModel.load_state_dict(torch.load(file_path, map_location=torch.device('cpu')))

newModel.eval()

NNSolver(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [4]:
    
input_data = torch.tensor([[1,5,6,7,17,2,0.9,0.9,0.9,1]])

#input_data = torch.tensor([[1,9,12,16,20,1.4,1,0.6,1,2]])
output_data = newModel(input_data)

In [5]:
print('Input: ', input_data)
print('Output: ', output_data)

Input:  tensor([[ 1.0000,  5.0000,  6.0000,  7.0000, 17.0000,  2.0000,  0.9000,  0.9000,
          0.9000,  1.0000]])
Output:  tensor([[10.0000, 15.0000, 17.0000, 19.0000, 21.0000,  0.6978,  2.0000,  1.9507,
          2.0000,  3.0000]], grad_fn=<CopySlices>)


In [6]:
import test_level as tl

output_data = output_data.detach().numpy().tolist()

tl.gT(output_data[0])
print('- - -')
tl.obj_fn(output_data[0])

[array([-1.59004211, -1.14501953, -4.25704193, -2.9822464 , -2.9822464 ,
       -1.71475983, -1.74835968, -0.49360657, -0.49360657,  1.37815857,
        1.37815857])]
- - -


130.65090663283374

In [12]:
DEVICE = torch.device("xpu" if torch.xpu.is_available() else "cpu")

print(DEVICE)

xpu


In [11]:
torch.xpu.is_available()

True

In [9]:
data.X


tensor([[ 2.0000,  4.0000,  9.0000,  ...,  2.7583,  2.7608,  2.4257],
        [ 0.0000,  3.0000,  7.0000,  ...,  3.3119,  2.9154,  0.2704],
        [ 9.0000, 11.0000, 15.0000,  ...,  1.4648,  3.6794,  0.1000],
        ...,
        [ 1.0000,  3.0000,  9.0000,  ...,  3.6602,  1.7535,  3.8391],
        [ 1.0000,  3.0000,  9.0000,  ...,  0.3673,  2.0807,  0.1000],
        [ 0.0000,  2.0000,  8.0000,  ...,  3.1508,  4.1285,  1.0117]])